# Задание №1 (easy)

In [1]:
import pandas
import sklearn.model_selection
import sklearn.ensemble
import sklearn.metrics

C:\Users\Artem\AppData\Local\Temp\ipykernel_5404\181903903.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
%%time

data = pandas.read_csv("../data/merc.csv")

x, y = data.drop(columns=["model", "price", "transmission", "fuelType"]), data["price"]
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, train_size=0.8, random_state=2)

model = sklearn.ensemble.RandomForestClassifier(n_jobs=8, random_state=2)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print(sklearn.metrics.r2_score(y_pred, y_test), '\n', pandas.DataFrame(zip(model.feature_names_in_, model.feature_importances_)))

0.860109097261408 
             0         1
0        year  0.047259
1     mileage  0.696300
2         tax  0.049054
3         mpg  0.166710
4  engineSize  0.040678
CPU times: total: 4min
Wall time: 47.7 s


Наиболее важной характеристикой оказался пробег (неудивительно, ведь чем больше автомобиль проехал, тем меньший ресурс у него остался, что также повышает риски поломок). Далее потребление топлива (что также весьма логично, ведь чем больше автомобиль потребляет топлива, тем больше будет денежных затрат на единицу пройденного расстояния). Размер двигателя и год производства примерно в равной степени влияют на цену. 

# Задание №2 (medium)

In [3]:
import os

In [4]:
%%time

data = pandas.DataFrame()
for path in os.listdir("../data/"):
    item = pandas.read_csv("../data/" + path)
    data = pandas.concat([data, item], ignore_index=True)
data = data.drop(columns=["tax(£)", "fuel type", "engine size", "mileage2", "fuel type2", "engine size2", "reference"])
data = data.dropna()
data = data.sample(len(data) // 100, random_state=2)  # 100k too much for my PC :)

# some data are stored weirdly
for i in data.index:
    if isinstance(data.loc[i, "mileage"], str):
        data.loc[i, "mileage"] = int(data.loc[i, "mileage"].split()[0].replace(',', ''))

# map nonnumeric to int
features_hm = dict()
for feature in ["model", "transmission", "fuelType"]:
    for index, key in enumerate(set(data[feature])):
        features_hm[key] = index

for i in data.index:
    for feature in ["model", "transmission", "fuelType"]:
        data.loc[i, feature] = features_hm[data.loc[i, feature]]


x, y = data.drop(columns=["price"]), data["price"].astype("float64")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, train_size=0.8, random_state=2)

model = sklearn.ensemble.RandomForestClassifier(n_jobs=8, random_state=2)

# default params: [100, "gini", None, "sqrt"]
params = {
    "n_estimators" : [1, 5, 10, 25, 50, 100, 150],
    "criterion"    : ["gini", "entropy", "log_loss"],
    "max_depth"    : [1, 5, 10, 25, 50, None],
    "max_features" : ["sqrt", "log2", None]
}
search = sklearn.model_selection.GridSearchCV(model, params)
search.fit(x_train, y_train)
search.best_estimator_

C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


CPU times: total: 12min 58s
Wall time: 7min 5s


RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=150,
                       n_jobs=8, random_state=2)

Я не смог придумать, чем можно объяснить такой результат, но получилось вот так.